# Aspects (plus) avancés pour la bioinformatique: exécution d'un programme local

Avec Python, on écrit souvent des programmes qui exécuteront un travail avec vos données sans recours à des outils externes mais on peut aussi utiliser Python pour utiliser notre poste de travail (que ce soit nu portable ou un serveur) pour exécuter des programmes spécialement dédiés à certaines fonctions qui sont hébergés sur cet ordinateur. Ça peut paraitre trivial mais il y a quand même des détails à noter pour que ça fonctionne bien :-)

## Un exemple: Rouler BLAST sur votre poste de travail. 

Le logiciel d'alignement de séquences <i>pairwise</i> NCBI BLAST est, et de loin, le logiciel le plus utilisé en biologie moléculaire. Rappel: une séquence inconnue (contenant des nucléotides ou des acides aminés) est comparée contre toutes les séquences contenues dans une base de données; suite à l'exécution de BLAST, celui-ci retourne les résultats de la recherche de similarité entre la séquence inconnue et des séquences connues contenues dans une base de données.

Un pré-requis absolu: le programme doit être soit disponible sur `$PATH`, soit accessible parce que vous connaissez sa localisatio ndans le système de fichier de votre poste de travail. DAns l'exemple que nous utiliserons, on assume que BLAST est sur `$PATH`. Quant à la nécessité d'avoir une base de données locales, on passera outre car BLAST à un paramétre, `-remote`, qui enverra notre reuête aux serveurs du NCBI pour utiliser leurs copies des bases de données. Dans notre exemple, nous utiliserons la base de données `refseq_protein` pour accélérer notre recherche.

Une ligne de commande BLAST est minimalement contruit de la manière suivante (il y a une quantité impressionnante de possibilités...):

```
#
# Disons que notre séquence en est une de protéines donc blastp
#
blastp -remote -query ma_sequence_aa.fa -out ma_sequence_aa.blast -db refseq_protein  
```

Mettre ça dans un script pourrait être intéressant: imaginons que vous voudriez tester l'effet de l'usage de diverses matrices de substitution sur les résultats de la recherche? Mettez vos crtitères dans une liste :-) 

Maintenant, comment interagir avec votre poste? Dans la littérature sur le sujet, il n'y a pas vraiment de consensus sur LA manière de faire car il existe par défaut dans une installation Python plusieurs façons de faire. Ici, nous utiliserons la méthode `subprocess` car elle nous permet des modes d'interaction intéressants pour interagir avec le poste effectuant le travail.



In [17]:
#
# La librairie qui fait travail
#
import os
import subprocess
import time

#
# Imaginons nos matrices, 
# de la moins vers la plus stringente
#
matrix = ["BLOSUM62","BLOSUM80","BLOSUM90"]

#
# Ou se trouve/mettre les fichiers
#
aFileIn = os.path.abspath("../z.misc_files/data_seq/h.sapiens.cttn.iso.a.aa.fa")
aFileOut = os.path.abspath("../z.misc_files/data_seq/h.sapiens.cttn.iso.a.aa.blast")

db = "refseq_protein"

cmd = ["blastp","-query",aFileIn,"-db",db]
#
# Si nous avions une DB locale pour execution locale, on mettra cette valeur à False
#
remote = True

if remote:
    cmd.append("-remote")
#
# On peut utiliser la méthode run ou la méthode Popen
#
# Je choisi d'ordinaire Popen car on a plus de controle sur ce qui se passe
# On construit un tableau avec les valeurs nécessaires pour faire rouler... 
#
# Juste une bonne pratique de vérifier que le fichier existe ;-)
#
if not os.path.exists(aFileIn):
  exit("File named "+aFileIn+" does not exists...")
else:
  for mat in matrix:
    tmp = list.copy(cmd)
    tmp.append("-matrix")
    tmp.append(mat)
    tmp.append("-out")
    tmp.append(aFileOut+"."+mat)
    #
    # On passe tmp comme liste des parametres pour BLAST et on dit de capturer
    # les sorties de la console (stdout) et les messages d'erreur (stderr) si
    # on termine avec un état non-zéro.
    #
    run = subprocess.Popen(tmp,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #
    # Ces variables vont contenir ce qui va sortir de BLAST. Si vous n'aviez pas 
    # spécifier un fichier de sortie, la variable out contiendrait votre résultat
    # sous forme d'un tableau, à gérer en conséquence.
    #
    out, errors = run.communicate()

    #print(out)
    #print(errors)
    tmp.clear()

    #
    # Important d'ajouter un délai entre les requetes pour ne pas se faire bloquer.
    #
    # 10 minutes, c'est peut etre beaucoup mais je sais que 
    # le NCBI ne me bloquera pas! 
    #
    time.sleep(600)

b''
b''
b''
b''
b''
b''
